In [1]:
%pip install --upgrade torch
%pip install transformers[torch]
%pip install --upgrade mlflow
%pip install --upgrade transformers
%pip install --upgrade datasets==2.9.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install azureml-mlflow==1.52.0

Note: you may need to restart the kernel to use updated packages.


In [47]:
# import os
# os.environ["DISABLE_MLFLOW_INTEGRATION"] = "TRUE"

was throwing error when included in the notebook : RuntimeError: MLflowCallback requires mlflow to be installed. Run `pip install mlflow`.

In [3]:
%pip install azureml-mlflow==1.52.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os 
import mlflow
import pandas as pd
import numpy as np
from datasets import load_dataset
import evaluate
from azureml.core import Workspace
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments,Trainer
import pickle
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential,AzureCliCredential 
from azure.ai.ml import MLClient

from azure.ai.ml.entities import AmlCompute
import time

subscription_id = '80c77c76-74ba-4c8c-8229-4c3b2957990c'
resource_group = 'sonata-test-rg'
workspace_name = 'sonata-test-ws'
credential = AzureCliCredential()
ws = Workspace(subscription_id, resource_group, workspace_name)
workspace_ml_client = MLClient(

        credential, subscription_id, resource_group, ws

    )

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import mlflow

In [7]:
EXPERIMENT_NAME = "Albert"
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='', creation_time=1688993909381, experiment_id='c181ad1e-a572-461c-ad0c-149da3813dfe', last_update_time=None, lifecycle_stage='active', name='Albert', tags={}>

In [8]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [9]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AlbertTokenizer,
    AlbertForSequenceClassification,
    AlbertModel,
    TrainingArguments,
    Trainer,
    EvalPrediction,
)

checkpoint = "albert-base-v2"
model = AlbertForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AlbertTokenizer.from_pretrained(checkpoint)
raw_ds= load_dataset("glue", "mrpc")
raw_ds['train'] = raw_ds['train'].shuffle().select(range(100))
raw_ds['test']= raw_ds['test'].shuffle().select(range(20))
raw_ds['validation']= raw_ds['validation'].shuffle().select(range(20))
metric = evaluate.load("glue", "mrpc")
dataset = raw_ds.map(
    lambda x: tokenizer(x["sentence1"], x["sentence2"], truncation=True),
    batched=True,
)
dataset = dataset.remove_columns(["sentence1", "sentence2", "idx"])
dataset = dataset.rename_column("label", "labels")
dataset = dataset.with_format("torch")

trainer_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")


def compute_metrics(eval_preds: EvalPrediction):
    x, y = eval_preds
    preds = np.argmax(x, -1)
    return metric.compute(predictions=preds, references=y)


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model 

In [10]:
import mlflow.pytorch

In [76]:
# mlflow.pytorch.save_model(model,path = "./savedmodel")

In [11]:
mlflow.transformers.save_model(
    transformers_model={"model": model, "tokenizer": tokenizer },
    path="./albertsavedmodel"
    # signature=signature,
    # input_example=data,
)

/tmp/ipykernel_12225/2231628459.py:1: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.29.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlflow.transformers.save_model(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [78]:
print(mlflow.__version__)

2.4.2


In [12]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [81]:
# with mlflow.start_run():
#     # registered_model_name="Albert"
#     model_local_path = os.path.abspath("./savedmodel")
#     mlflow.register_model(f"file://{model_local_path}", "AlbertModel")

Registered model 'AlbertModel' already exists. Creating a new version of this model...
2023/07/14 10:26:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: AlbertModel, version 4
Created version '4' of model 'AlbertModel'.


In [13]:
with mlflow.start_run():
    # registered_model_name="Albert"
    model_local_path = os.path.abspath("./albertsavedmodel")
    mlflow.register_model(f"file://{model_local_path}", "AlbertModel")

Registered model 'AlbertModel' already exists. Creating a new version of this model...
2023/07/14 12:25:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: AlbertModel, version 7
Created version '7' of model 'AlbertModel'.


In [83]:
# mlmodel = mlflow.pyfunc.load_model("savedmodel2")
# mlmodel = mlflow.transformers.load_model("savedmodel")

2023/07/14 10:26:36 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.2, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2023/07/14 10:26:37 WARNING mlflow.transformers: Could not specify device parameter for this pipeline type


In [15]:
mlmodel = mlflow.transformers.load_model("albertsavedmodel")

/tmp/ipykernel_12225/804854849.py:1: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.29.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlmodel = mlflow.transformers.load_model("albertsavedmodel")
2023/07/14 12:26:16 WARNING mlflow.transformers: Could not specify device parameter for this pipeline type


In [16]:
mlmodel.model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [17]:
%pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [18]:
trainer = Trainer(
    model=mlmodel.model,
    tokenizer=tokenizer,
    args=trainer_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"]
    # test_dataset=dataset["test"]
)

result = trainer.train()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.629952,0.700000,0.823529
2,No log,0.581840,0.700000,0.823529
3,No log,0.510912,0.700000,0.823529


In [ ]:
#retrieve trained model

trained_model = trainer.model
trained_model.config

In [10]:
mlflow.end_run()